In [2]:
import pandas as pd
import numpy as np
import requests
import time
import pandas as pd
from sqlalchemy import create_engine


from config import weather_api_key

In [3]:
wine_file = "Resources/winemag-data-130k-v2.csv"
wine_df = pd.read_csv(wine_file)
wine_df

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [4]:
cleaned_df = wine_df[["variety", "points", "price", "country", "province", "region_1"]]
cleaned_df = cleaned_df.dropna()
cleaned_df


,variety,points,price,country,province,region_1
2,Pinot Gris,87,14.0,US,Oregon,Willamette Valley
3,Riesling,87,13.0,US,Michigan,Lake Michigan Shore
4,Pinot Noir,87,65.0,US,Oregon,Willamette Valley
5,Tempranillo-Merlot,87,15.0,Spain,Northern Spain,Navarra
6,Frappato,87,16.0,Italy,Sicily & Sardinia,Vittoria
...,...,...,...,...,...,...
129965,Pinot Gris,90,28.0,France,Alsace,Alsace
129967,Pinot Noir,90,75.0,US,Oregon,Oregon
129968,Gewürztraminer,90,30.0,France,Alsace,Alsace
129969,Pinot Gris,90,32.0,France,Alsace,Alsace


In [5]:
province = cleaned_df[["country", "province", "region_1"]]
provinces_cleaned = province.drop_duplicates()
provinces_cleaned

,country,province,region_1
2,US,Oregon,Willamette Valley
3,US,Michigan,Lake Michigan Shore
5,Spain,Northern Spain,Navarra
6,Italy,Sicily & Sardinia,Vittoria
7,France,Alsace,Alsace
...,...,...,...
126567,US,California,Sonoma-Napa-Lake
126987,US,California,Sonoma County-Santa Barbara County
129331,Italy,Northeastern Italy,Del Veneto
129411,Italy,Veneto,Bardolino Superiore


In [21]:
lat = []
lng = []
max_temp = []
humid = []
cloudy = []
winds = []
country = []
date = []
city_weather = []

#variables for records
record_num = 0
Set = 1

print("Beginning Data Retrieval")
print("----------------------------")

#loop through cities retreiving data for each city
for i in provinces_cleaned["province"]:
    time.sleep(1)
    
    #check to see if there is data for city
    try:
        query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={i}&appid={weather_api_key}&units=imperial")
        response = requests.get(query_url).json()

        if record_num == 50:
            Set += 1
            record_num = 0
        else:
            record_num += 1 
            
        #print record saying which city is being processed
        print(f"Processing Record {record_num} of Set {Set} | {i}")
        
        #get data for city 
        cities_lng = response['coord']['lon']
        cities_lat = response['coord']["lat"]
        cities_max_temp = response['main']['temp_max']
        cities_humid = response['main']['humidity']
        cities_cloudy = response['clouds']['all']
        cities_winds = response['wind']['speed']
        cities_country = response['sys']['country']
        cities_date = response['dt']

        #Create dictionary with data gathered
        city_weather.append({
        "City": i,
        "Lat": cities_lat,
        "Lng": cities_lng,
        "Max Temp": cities_max_temp,
        "Humidity": cities_humid,
        "Cloudiness": cities_cloudy,
        "Wind Speed": cities_winds,
        "Country": cities_country,
        "Date": cities_date
        })

    except:
        print(f"City {i} not found... Sorry!")
        pass
    12
print("Complete!")

Beginning Data Retrieval
----------------------------
Processing Record 1 of Set 1 | Oregon
Processing Record 2 of Set 1 | Michigan
Processing Record 3 of Set 1 | Northern Spain
City Northern Spain not found... Sorry!
Processing Record 4 of Set 1 | Sicily & Sardinia
City Sicily & Sardinia not found... Sorry!
Processing Record 5 of Set 1 | Alsace


KeyboardInterrupt: 

In [23]:
lat = []
lng = []
max_temp = []
min_temp = []
temp = []
humid = []
cloudy = []
winds = []
country = []
date = []
city_weather = []

#variables for records
record_num = 0
Set = 1

print("Beginning Data Retrieval")
print("----------------------------")

#loop through cities retreiving data for each city
for index, row in provinces_cleaned.iterrows():
    time.sleep(1)
    
    #check to see if there is data for city
    try:
        i = row["region_1"]
        query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={i}&appid={weather_api_key}&units=imperial")
        response = requests.get(query_url).json()

        if record_num == 50:
            Set += 1
            record_num = 0
        else:
            record_num += 1 
            
        #print record saying which city is being processed
        print(f"Processing Record {record_num} of Set {Set} | {i}")
        
        #get data for city 
        cities_lng = response['coord']['lon']
        cities_lat = response['coord']["lat"]
        cities_temp= response['main']['temp']
        cities_max_temp = response['main']['temp_max']
        cities_min_temp = response['main']['temp_min']
        cities_humid = response['main']['humidity']
        cities_country = response['sys']['country']
  

        #Create dictionary with data gathered
        city_weather.append({
        "City": i,
        "Lat": cities_lat,
        "Lng": cities_lng,
        "Temp": cities_temp,
        "Max Temp": cities_max_temp,
        "Min Temp": cities_min_temp,
        "Humidity": cities_humid,
        "Cloudiness": cities_cloudy,
        "Wind Speed": cities_winds,
        "Country": cities_country,
        "Date": cities_date
        })

    except:
        print(f"region {i} not found... Sorry!")

        try:
            a = row["province"]
            query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={a}&appid={weather_api_key}&units=imperial")
            response = requests.get(query_url).json()

            #get data for city 
            cities_lng = response['coord']['lon']
            cities_lat = response['coord']["lat"]
            cities_temp= response['main']['temp']
            cities_max_temp = response['main']['temp_max']
            cities_min_temp = response['main']['temp_min']
            cities_humid = response['main']['humidity']
            cities_country = response['sys']['country']


            #Create dictionary with data gathered
            city_weather.append({
            "City": i,
            "Lat": cities_lat,
            "Lng": cities_lng,
            "Temp": cities_temp,
            "Max Temp": cities_max_temp,
            "Min Temp": cities_min_temp,
            "Humidity": cities_humid,
            "Cloudiness": cities_cloudy,
            "Wind Speed": cities_winds,
            "Country": cities_country,
            "Date": cities_date
            })
            

        except:
            print(f"province {a} not found... Sorry!")
            
            try:
                
                b = row["country"]
                query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={b}&appid={weather_api_key}&units=imperial")
                response = requests.get(query_url).json()

                #get data for city 
                cities_lng = response['coord']['lon']
                cities_lat = response['coord']["lat"]
                cities_temp= response['main']['temp']
                cities_max_temp = response['main']['temp_max']
                cities_min_temp = response['main']['temp_min']
                cities_humid = response['main']['humidity']
                cities_country = response['sys']['country']


                #Create dictionary with data gathered
                city_weather.append({
                "City": i,
                "Lat": cities_lat,
                "Lng": cities_lng,
                "Temp": cities_temp,
                "Max Temp": cities_max_temp,
                "Min Temp": cities_min_temp,
                "Humidity": cities_humid,
                "Cloudiness": cities_cloudy,
                "Wind Speed": cities_winds,
                "Country": cities_country,
                "Date": cities_date
                })
                

            except:      
                print(f"country {b} not found... Sorry!")
                pass
    12
print("Complete!")

Beginning Data Retrieval
----------------------------
Processing Record 1 of Set 1 | Willamette Valley
region Willamette Valley not found... Sorry!
Processing Record 2 of Set 1 | Lake Michigan Shore
region Lake Michigan Shore not found... Sorry!
Processing Record 3 of Set 1 | Navarra
Processing Record 4 of Set 1 | Vittoria
Processing Record 5 of Set 1 | Alsace
Processing Record 6 of Set 1 | Napa Valley
Processing Record 7 of Set 1 | Alexander Valley
region Alexander Valley not found... Sorry!
Processing Record 8 of Set 1 | Central Coast
Processing Record 9 of Set 1 | Cafayate
Processing Record 10 of Set 1 | Mendoza
Processing Record 11 of Set 1 | Ribera del Duero
region Ribera del Duero not found... Sorry!
province Northern Spain not found... Sorry!
Processing Record 12 of Set 1 | Virginia
Processing Record 13 of Set 1 | Oregon
Processing Record 14 of Set 1 | Sicilia
Processing Record 15 of Set 1 | Paso Robles
Processing Record 16 of Set 1 | Sonoma Coast
region Sonoma Coast not found..

Processing Record 7 of Set 3 | Montsant
region Montsant not found... Sorry!
Processing Record 8 of Set 3 | Arbois
Processing Record 9 of Set 3 | Barossa Valley
region Barossa Valley not found... Sorry!
Processing Record 10 of Set 3 | Tupungato
Processing Record 11 of Set 3 | Mendocino County
Processing Record 12 of Set 3 | Côte de Nuits-Villages
region Côte de Nuits-Villages not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 13 of Set 3 | Langhe
region Langhe not found... Sorry!
Processing Record 14 of Set 3 | Prosecco di Valdobbiadene
region Prosecco di Valdobbiadene not found... Sorry!
Processing Record 15 of Set 3 | Prosecco di Conegliano
region Prosecco di Conegliano not found... Sorry!
Processing Record 16 of Set 3 | Languedoc
Processing Record 17 of Set 3 | Coteaux Varois en Provence
region Coteaux Varois en Provence not found... Sorry!
Processing Record 18 of Set 3 | Cava
Processing Record 19 of Set 3 | Rutherglen
Processing Record 20 of Set 3 | Barossa


Processing Record 7 of Set 5 | Mâcon-Villages
region Mâcon-Villages not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 8 of Set 5 | Texas
Processing Record 9 of Set 5 | Côte Chalonnaise
region Côte Chalonnaise not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 10 of Set 5 | Chorey-lès-Beaune
region Chorey-lès-Beaune not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 11 of Set 5 | Calchaquí Valley
region Calchaquí Valley not found... Sorry!
province Other not found... Sorry!
Processing Record 12 of Set 5 | Coteaux Bourguignons
region Coteaux Bourguignons not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 13 of Set 5 | Saint-Véran
Processing Record 14 of Set 5 | Barbaresco
Processing Record 15 of Set 5 | South Eastern Australia
region South Eastern Australia not found... Sorry!
province Australia Other not found... Sorry!
Processing Record 16 of Set 5 | Temecula Valley
region Temecula Vall

Processing Record 4 of Set 7 | Vigneti delle Dolomiti
region Vigneti delle Dolomiti not found... Sorry!
province Northeastern Italy not found... Sorry!
Processing Record 5 of Set 7 | Montefalco Rosso
region Montefalco Rosso not found... Sorry!
province Central Italy not found... Sorry!
Processing Record 6 of Set 7 | Sagrantino di Montefalco
region Sagrantino di Montefalco not found... Sorry!
province Central Italy not found... Sorry!
Processing Record 7 of Set 7 | Umbria
Processing Record 8 of Set 7 | Faro
Processing Record 9 of Set 7 | Chénas
Processing Record 10 of Set 7 | Jurançon
Processing Record 11 of Set 7 | Saumur
Processing Record 12 of Set 7 | Alella
Processing Record 13 of Set 7 | Bardolino Chiaretto
region Bardolino Chiaretto not found... Sorry!
Processing Record 14 of Set 7 | Murgia
Processing Record 15 of Set 7 | Bolgheri Superiore
region Bolgheri Superiore not found... Sorry!
Processing Record 16 of Set 7 | Bolgheri Sassicaia
region Bolgheri Sassicaia not found... Sorry!

Processing Record 2 of Set 9 | Niagara Peninsula
region Niagara Peninsula not found... Sorry!
Processing Record 3 of Set 9 | Molise
Processing Record 4 of Set 9 | New Jersey
Processing Record 5 of Set 9 | Maury Sec
region Maury Sec not found... Sorry!
Processing Record 6 of Set 9 | Côtes du Roussillon Villages Caramany
region Côtes du Roussillon Villages Caramany not found... Sorry!
Processing Record 7 of Set 9 | Redwood Valley
Processing Record 8 of Set 9 | Friuli Aquileia
region Friuli Aquileia not found... Sorry!
province Northeastern Italy not found... Sorry!
Processing Record 9 of Set 9 | Ile de Beauté
region Ile de Beauté not found... Sorry!
province France Other not found... Sorry!
Processing Record 10 of Set 9 | Cucamonga Valley
region Cucamonga Valley not found... Sorry!
Processing Record 11 of Set 9 | Pays d'Oc
region Pays d'Oc not found... Sorry!
Processing Record 12 of Set 9 | Gard
Processing Record 13 of Set 9 | Chianti Montespertoli
region Chianti Montespertoli not found.

...

Processing Record 19 of Set 17 | Mâcon
Processing Record 20 of Set 17 | Lirac
Processing Record 21 of Set 17 | Coteaux du Layon
region Coteaux du Layon not found... Sorry!
province Loire Valley not found... Sorry!
Processing Record 22 of Set 17 | Pacherenc du Vic Bilh
region Pacherenc du Vic Bilh not found... Sorry!
province Southwest France not found... Sorry!
Processing Record 23 of Set 17 | Kelsey Bench-Lake County
region Kelsey Bench-Lake County not found... Sorry!
Processing Record 24 of Set 17 | San Lucas
Processing Record 25 of Set 17 | British Columbia
Processing Record 26 of Set 17 | Manchuela
region Manchuela not found... Sorry!
province Central Spain not found... Sorry!
Processing Record 27 of Set 17 | Biferno Rosso
region Biferno Rosso not found... Sorry!
province Southern Italy not found... Sorry!
Processing Record 28 of Set 17 | Sonoma County-Lake County
region Sonoma County-Lake County not found... Sorry!
Processing Record 29 of Set 17 | France
Processing Record 30 of Se

province Burgundy not found... Sorry!
Processing Record 5 of Set 19 | Sonoma
Processing Record 6 of Set 19 | Mount Barker
Processing Record 7 of Set 19 | Corton-Pougets
region Corton-Pougets not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 8 of Set 19 | Cérons
Processing Record 9 of Set 19 | Gioia del Colle
Processing Record 10 of Set 19 | Vin Santo del Chianti
region Vin Santo del Chianti not found... Sorry!
Processing Record 11 of Set 19 | Kentucky
Processing Record 12 of Set 19 | Prince Edward County
Processing Record 13 of Set 19 | Extremadura
Processing Record 14 of Set 19 | Vino da Tavola
region Vino da Tavola not found... Sorry!
province Italy Other not found... Sorry!
Processing Record 15 of Set 19 | Southern Fleurieu
region Southern Fleurieu not found... Sorry!
Processing Record 16 of Set 19 | Clements Hills
region Clements Hills not found... Sorry!
Processing Record 17 of Set 19 | Vinos de Madrid
region Vinos de Madrid not found... Sorry!
province C

province Central Italy not found... Sorry!
Processing Record 37 of Set 20 | Erbaluce di Caluso
region Erbaluce di Caluso not found... Sorry!
Processing Record 38 of Set 20 | Mamertino
region Mamertino not found... Sorry!
province Sicily & Sardinia not found... Sorry!
Processing Record 39 of Set 20 | Mâcon Solutré
region Mâcon Solutré not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 40 of Set 20 | Civitella d'Agliano
Processing Record 41 of Set 20 | Landes
Processing Record 42 of Set 20 | Mâcon-Davayé
region Mâcon-Davayé not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 43 of Set 20 | Pedernal Valley
region Pedernal Valley not found... Sorry!
province Other not found... Sorry!
Processing Record 44 of Set 20 | Breganze
Processing Record 45 of Set 20 | Tehama County
Processing Record 46 of Set 20 | Siracusa
Processing Record 47 of Set 20 | Vin de Pays du Jardin de la France
region Vin de Pays du Jardin de la France not found... Sorry!
p

Processing Record 19 of Set 22 | Hilltops
region Hilltops not found... Sorry!
Processing Record 20 of Set 22 | Polk County
Processing Record 21 of Set 22 | Tuscany
Processing Record 22 of Set 22 | Barossa-Langhorne Creek
region Barossa-Langhorne Creek not found... Sorry!
Processing Record 23 of Set 22 | Currency Creek-McLaren Vale
region Currency Creek-McLaren Vale not found... Sorry!
Processing Record 24 of Set 22 | Coteaux du Lyonnais
region Coteaux du Lyonnais not found... Sorry!
Processing Record 25 of Set 22 | Sardinia
Processing Record 26 of Set 22 | Gippsland
region Gippsland not found... Sorry!
Processing Record 27 of Set 22 | Strathbogie Ranges
region Strathbogie Ranges not found... Sorry!
Processing Record 28 of Set 22 | Colline Teramane
region Colline Teramane not found... Sorry!
province Central Italy not found... Sorry!
Processing Record 29 of Set 22 | Vino de la Tierra Ribera del Gállego-Cinco Villas
region Vino de la Tierra Ribera del Gállego-Cinco Villas not found... So

province Southern Italy not found... Sorry!
Processing Record 1 of Set 24 | Sonoma-Napa-Monterey
region Sonoma-Napa-Monterey not found... Sorry!
Processing Record 2 of Set 24 | Collioure
Processing Record 3 of Set 24 | Saint Guilhem le Désert
region Saint Guilhem le Désert not found... Sorry!
Processing Record 4 of Set 24 | Catalanesca del Monte Somma
region Catalanesca del Monte Somma not found... Sorry!
province Southern Italy not found... Sorry!
Processing Record 5 of Set 24 | Périgord
Processing Record 6 of Set 24 | Hawaii
Processing Record 7 of Set 24 | San Gimignano
Processing Record 8 of Set 24 | Bourgogne Vézelay
region Bourgogne Vézelay not found... Sorry!
province Burgundy not found... Sorry!
Processing Record 9 of Set 24 | Vino de Calidad de Valtiendas
region Vino de Calidad de Valtiendas not found... Sorry!
province Northern Spain not found... Sorry!
Processing Record 10 of Set 24 | Mitterberg
Processing Record 11 of Set 24 | Rivesaltes Rosé
region Rivesaltes Rosé not found

In [27]:
weather = pd.DataFrame(city_weather)
weather.to_csv("./Resources/output.csv", index_label = "City ID")
weather

,City,Lat,Lng,Temp,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Willamette Valley,44.0001,-120.5014,55.24,60.03,55.22,39,0,4.61,US,1632192236
1,Lake Michigan Shore,44.2503,-85.5003,71.49,71.67,68.50,83,0,4.61,US,1632192236
2,Navarra,42.7500,-1.6667,51.13,53.53,51.12,93,0,4.61,ES,1632192236
3,Vittoria,36.9537,14.5332,73.92,73.92,73.92,78,0,4.61,IT,1632192236
4,Alsace,48.5000,7.5000,53.31,53.74,53.31,87,0,4.61,FR,1632192236
...,...,...,...,...,...,...,...,...,...,...,...
1201,Sonoma-Napa-Lake,38.3004,-76.5074,68.72,73.08,62.42,73,0,4.61,US,1632192236
1202,Sonoma County-Santa Barbara County,38.3004,-76.5074,68.72,73.08,62.42,73,0,4.61,US,1632192236
1203,Del Veneto,32.1840,-96.8847,85.28,88.12,82.06,37,0,4.61,US,1632192236
1204,Bardolino Superiore,45.5000,11.7500,60.91,61.02,58.75,83,0,4.61,IT,1632192236
